In [49]:
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import KMeans
import numpy as np
import math
from scipy.stats import skew
from scipy.stats import tvar
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from scipy.special import sici
from scipy.stats import median_absolute_deviation
import collections
from past.builtins import xrange
import datetime
from datetime import timedelta
from pandas import DataFrame
from scipy.fftpack import fft, ifft,dct
import pywt
import scipy.stats as stats
from sklearn.metrics import classification_report, accuracy_score, make_scorer

import warnings
warnings.filterwarnings("ignore")

In [50]:
insulin_data1=pd.read_csv('InsulinData.csv',low_memory=False, parse_dates = [['Date', 'Time']])[['Date_Time', 'BWZ Carb Input (grams)']]
cgm_data1=pd.read_csv('CGMData.csv',low_memory=False,parse_dates = [['Date', 'Time']])

In [51]:
def trimData(insulin_data, cgm_data):
    min_insulin_time = min(insulin_data['Date_Time'])
    min_cgm_time = min(cgm_data['Date_Time'])
    min_timestamp = max(min_insulin_time, min_cgm_time)
    max_insulin_time = max(insulin_data['Date_Time'])
    max_cgm_time = max(cgm_data['Date_Time'])
    max_timestamp = min(max_insulin_time, max_cgm_time)
    insulin_trimmed = insulin_data[(insulin_data['Date_Time'] >= min_timestamp)]
    insulin_trimmed = insulin_trimmed.reset_index(drop=True)
    insulin_trimmed = insulin_data[(insulin_data['Date_Time'] <= max_timestamp)]
    insulin_trimmed = insulin_trimmed.reset_index(drop=True)
    cgm_trimmed = cgm_data[(cgm_data['Date_Time'] >= min_timestamp)]
    cgm_trimmed = cgm_trimmed.reset_index(drop=True)
    cgm_trimmed = cgm_data[(cgm_data['Date_Time'] <= max_timestamp)]
    cgm_trimmed = cgm_trimmed.reset_index(drop=True)
    return insulin_trimmed, cgm_trimmed

In [52]:
insulin_data1_trimmed, cgm_data1_trimmed = trimData(insulin_data1, cgm_data1)

In [53]:
Bool_carbinput_data1=pd.notnull(insulin_data1_trimmed['BWZ Carb Input (grams)'])
insuline_data_ext1=insulin_data1_trimmed[Bool_carbinput_data1]
insuline_data_ext1 = insuline_data_ext1[insuline_data_ext1['BWZ Carb Input (grams)'] != 0].sort_values(by=['Date_Time'], ignore_index = True)
insuline_data_ext1 = insuline_data_ext1.reset_index(drop=True)
print(insuline_data_ext1)

              Date_Time  BWZ Carb Input (grams)
0   2017-07-25 10:21:19                    91.0
1   2017-07-25 10:39:46                    58.0
2   2017-07-25 18:31:40                   115.0
3   2017-07-26 09:27:16                    72.0
4   2017-07-26 11:24:52                    20.0
..                  ...                     ...
742 2018-02-11 16:27:04                    40.0
743 2018-02-11 18:14:37                     8.0
744 2018-02-11 20:33:18                    71.0
745 2018-02-12 02:30:55                    15.0
746 2018-02-12 09:15:45                    34.0

[747 rows x 2 columns]


In [54]:
insulin_data_carblist= (insuline_data_ext1['BWZ Carb Input (grams)']).tolist()
max_carb_val=max(insulin_data_carblist)
min_carb_val=min(insulin_data_carblist)
bin_size=int(max_carb_val+min_carb_val)//20
bins=range(bin_size)
dict1={}
for i in range(1,len(bins)+1):
  if i ==1:
    dict1[i]=min_carb_val+20
  else:
    dict1[i]=dict1[i-1]+20 

In [55]:
print(insulin_data_carblist)
print(dict1)
tot_bins=[]
for i in dict1.keys():
  list_bin=[]
  for val in insulin_data_carblist:
    if val>=dict1[i]-20 and val<dict1[i]:
      list_bin.append(val)
      #print(val,i)
  tot_bins.append(list_bin)  
print(tot_bins)
print(len(tot_bins))  
    

[91.0, 58.0, 115.0, 72.0, 20.0, 63.0, 60.0, 24.0, 47.0, 54.0, 73.0, 70.0, 55.0, 60.0, 112.0, 24.0, 27.0, 80.0, 24.0, 51.0, 15.0, 122.0, 19.0, 24.0, 45.0, 90.0, 40.0, 11.0, 40.0, 24.0, 72.0, 80.0, 81.0, 37.0, 39.0, 40.0, 50.0, 48.0, 25.0, 38.0, 80.0, 10.0, 61.0, 30.0, 50.0, 20.0, 28.0, 8.0, 69.0, 41.0, 31.0, 47.0, 30.0, 40.0, 15.0, 67.0, 81.0, 60.0, 69.0, 38.0, 26.0, 28.0, 62.0, 22.0, 20.0, 10.0, 63.0, 20.0, 40.0, 30.0, 16.0, 75.0, 11.0, 60.0, 42.0, 38.0, 59.0, 35.0, 60.0, 24.0, 59.0, 63.0, 16.0, 30.0, 40.0, 26.0, 10.0, 46.0, 23.0, 45.0, 45.0, 20.0, 40.0, 50.0, 10.0, 35.0, 68.0, 12.0, 15.0, 26.0, 20.0, 45.0, 18.0, 27.0, 30.0, 85.0, 26.0, 129.0, 24.0, 35.0, 75.0, 51.0, 88.0, 23.0, 26.0, 30.0, 45.0, 43.0, 17.0, 52.0, 83.0, 15.0, 50.0, 40.0, 57.0, 102.0, 60.0, 53.0, 40.0, 9.0, 58.0, 100.0, 10.0, 36.0, 35.0, 85.0, 92.0, 45.0, 37.0, 85.0, 18.0, 20.0, 90.0, 40.0, 67.0, 110.0, 120.0, 80.0, 25.0, 33.0, 21.0, 35.0, 45.0, 23.0, 83.0, 40.0, 9.0, 55.0, 91.0, 71.0, 40.0, 22.0, 25.0, 30.0, 25.0, 55.0

In [56]:
def bin_dec(x):
  list1 = list(dict1.values())
  if x<=list1[0]:
    return 1
  elif x<=list1[1]:
    return 2
  elif x<=list1[2]:
    return 3
  elif x<=list1[3]:
    return 4
  elif x<=list1[4]:
    return 5
  elif x<=list1[5]:
    return 6

In [57]:
def extract_meal_cgm_timestamps(meal_dataframe):
    meal_data = []
    time_val = []
    time_bin = []
    meal_len = len(meal_dataframe)-1
    for row in range(0,meal_len):
        new_time = meal_dataframe.at[row, 'Date_Time'] + timedelta(hours = 2)
        if(new_time > meal_dataframe.at[row+1, 'Date_Time']):
           continue
        else:
             meal_data.append(meal_dataframe.at[row, 'Date_Time'])
             time_bin.append(bin_dec(meal_dataframe.at[row, 'BWZ Carb Input (grams)']))
             time_val.append(meal_dataframe.at[row, 'BWZ Carb Input (grams)'],)
    meal_dataframe_fin = pd.DataFrame(meal_data)
    meal_dataframe_fin['bins'] = time_bin
    meal_dataframe_fin['val'] = time_val
    
    return meal_dataframe_fin  

In [58]:
meal_data1 = extract_meal_cgm_timestamps(insuline_data_ext1)
print(meal_data1)

                      0  bins    val
0   2017-07-25 10:39:46   3.0   58.0
1   2017-07-25 18:31:40   6.0  115.0
2   2017-07-26 12:48:41   3.0   63.0
3   2017-07-26 19:15:06   3.0   60.0
4   2017-07-27 05:45:51   2.0   24.0
..                  ...   ...    ...
598 2018-02-11 10:02:19   1.0   10.0
599 2018-02-11 12:43:23   2.0   27.0
600 2018-02-11 18:14:37   1.0    8.0
601 2018-02-11 20:33:18   4.0   71.0
602 2018-02-12 02:30:55   1.0   15.0

[603 rows x 3 columns]


In [59]:
cgm_data1_trimmed['Sensor Glucose (mg/dL)'] = cgm_data1_trimmed['Sensor Glucose (mg/dL)'].interpolate(method = 'linear')

In [60]:
cgm_data1_trimmed = cgm_data1_trimmed[['Date_Time','Sensor Glucose (mg/dL)']]
cgm_data1_trimmed = cgm_data1_trimmed.reindex(index=cgm_data1_trimmed.index[::-1])
cgm_data1_trimmed = cgm_data1_trimmed.reset_index(drop=True)

In [61]:
def meal_cgm_extraction(meal_cgm_dataframe, cgmdata):
    list1 = ['cgm_val'+str(x) for x in range(30)]
    meal_data = pd.DataFrame(columns = list1)
    for id in meal_cgm_dataframe.index:
        dict1 = dict()
        data_sets = cgmdata[cgmdata['Date_Time'] >= meal_cgm_dataframe[0][id]]
        data_set_list = list(cgmdata.loc[data_sets.index[0]-6: data_sets.index[0]+23, 'Sensor Glucose (mg/dL)'].values)
        cgm_list=[]
        for id1, cgm_val in enumerate(data_set_list):
            cgm_list.append(cgm_val)
            for cgm in cgm_list:
              dict1[list1[id1]] = cgm_val
        meal_data = meal_data.append(dict1, ignore_index = True)
    meal_data['bins']=list(meal_cgm_dataframe['bins'])
    return meal_data

In [62]:
cgm_meal1 = meal_cgm_extraction(meal_data1, cgm_data1_trimmed)
meal_cgm_master = cgm_meal1.reset_index(drop=True)
meal_cgm_master_list = meal_cgm_master.values.tolist()
meal_cgm_master

,cgm_val0,cgm_val1,cgm_val2,cgm_val3,cgm_val4,cgm_val5,cgm_val6,cgm_val7,cgm_val8,cgm_val9,cgm_val10,cgm_val11,cgm_val12,cgm_val13,cgm_val14,cgm_val15,cgm_val16,cgm_val17,cgm_val18,cgm_val19,cgm_val20,cgm_val21,cgm_val22,cgm_val23,cgm_val24,cgm_val25,cgm_val26,cgm_val27,cgm_val28,cgm_val29,bins
0,314.000000,310.000000,309.000000,311.000000,311.000000,311.0,312.000000,312.000000,316.000000,324.000000,319.000000,315.000000,314.000000,316.000000,314.000000,310.000000,298.000000,293.000000,279.000000,278.000000,281.000000,289.000000,294.0,290.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0
1,58.000000,59.000000,63.000000,71.000000,81.000000,102.0,131.000000,140.000000,147.000000,153.000000,159.000000,163.000000,159.000000,153.000000,151.000000,155.000000,163.000000,164.000000,165.000000,166.000000,168.000000,164.000000,169.0,178.0,184.0,190.0,195.0,198.0,203.0,196.0,6.0
2,304.000000,292.000000,281.000000,268.000000,259.000000,255.0,248.000000,241.000000,231.000000,220.000000,222.000000,222.000000,222.000000,223.000000,224.000000,228.000000,233.000000,243.000000,256.000000,262.000000,270.000000,277.000000,274.0,269.0,267.0,267.0,274.0,284.0,283.0,278.0,3.0
3,40.000000,40.000000,40.000000,40.000000,60.000000,71.0,83.000000,87.000000,100.000000,112.000000,121.000000,125.000000,130.000000,132.000000,123.000000,112.000000,89.000000,77.000000,69.000000,66.000000,67.000000,71.000000,75.0,74.0,72.0,70.0,67.0,74.0,77.0,81.0,3.0
4,212.000000,210.000000,204.000000,200.000000,199.000000,201.0,201.000000,194.000000,188.000000,183.000000,181.000000,176.000000,170.000000,168.000000,169.000000,172.000000,179.000000,192.000000,203.000000,205.000000,210.000000,213.000000,212.0,216.0,213.0,210.0,210.0,209.0,210.0,209.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,57.000000,58.000000,62.000000,74.000000,104.000000,110.0,126.000000,133.000000,135.000000,141.000000,144.000000,137.000000,137.000000,148.000000,147.000000,136.000000,135.000000,136.000000,142.000000,168.000000,168.000000,174.000000,207.0,209.0,206.0,197.0,185.0,191.0,197.0,202.0,1.0
599,209.000000,219.000000,221.000000,221.000000,216.000000,217.0,215.000000,211.000000,201.000000,200.000000,198.000000,196.000000,194.000000,200.000000,220.000000,207.000000,205.000000,194.000000,186.000000,177.000000,180.000000,184.000000,180.0,190.0,196.0,203.0,206.0,212.0,215.0,208.0,2.0
600,169.176471,167.941176,166.705882,165.470588,164.235294,163.0,161.764706,160.529412,159.294118,158.058824,156.823529,155.588235,154.352941,153.117647,151.882353,150.647059,149.411765,148.176471,146.941176,145.705882,144.470588,143.235294,142.0,145.0,147.0,137.0,128.0,123.0,106.0,107.0,1.0
601,106.000000,107.000000,100.000000,104.000000,123.000000,132.0,129.000000,127.000000,120.000000,132.000000,138.000000,145.000000,160.000000,171.000000,178.000000,182.000000,180.000000,168.000000,166.000000,167.000000,169.000000,176.000000,173.0,166.0,162.0,189.0,201.0,200.0,194.0,177.0,4.0


In [63]:
def diff_max_min_cgm(meal_list,row):
  max_cgm_val = max(meal_list[row])
  min_cgm_val = min(meal_list[row])
  result = max_cgm_val - min_cgm_val
  return (result)

difference_list_meal = []
for row in range (len(meal_cgm_master_list)):
  difference = diff_max_min_cgm(meal_cgm_master_list,row)
  difference_list_meal.append(difference)
difference_df_meal = pd.DataFrame(difference_list_meal) 
difference_df_meal

,0
0,321.000000
1,197.000000
2,301.000000
3,129.000000
4,214.000000
...,...
598,208.000000
599,219.000000
600,168.176471
601,197.000000


In [64]:
def skew_c(meal_list,row):
  result = skew(meal_list[row])
  return (result)

skew_list_meal = []
for row in range (len(meal_cgm_master_list)):
  skew_val = skew_c(meal_cgm_master_list,row)
  skew_list_meal.append(skew_val)
skew_df_meal = pd.DataFrame(skew_list_meal) 
skew_df_meal

,0
0,NaN
1,-1.178143
2,-3.553611
3,-0.085566
4,-4.051013
...,...
598,-0.781161
599,-4.383150
600,-3.418930
601,-1.558934


In [65]:
def median_absolute_deviation_c(meal_list,row):
  result = median_absolute_deviation(meal_list[row])
  return (result)

median_absolute_deviation_list_meal = []
for row in range (len(meal_cgm_master_list)):
  median_absolute_deviation_val = median_absolute_deviation_c(meal_cgm_master_list,row)
  median_absolute_deviation_list_meal.append(median_absolute_deviation_val)
median_absolute_deviation_df_meal = pd.DataFrame(median_absolute_deviation_list_meal) 
median_absolute_deviation_df_meal

,0
0,NaN
1,28.169400
2,28.169400
3,19.273800
4,13.343400
...,...
598,45.960600
599,13.343400
600,12.820129
601,34.099800


In [66]:
def tvar_c(meal_list,row):
  result = tvar(meal_list[row])
  return (result)

tvar_list_meal = []
for row in range (len(meal_cgm_master_list)):
  tvar_val = tvar_c(meal_cgm_master_list,row)
  tvar_list_meal.append(tvar_val)
tvar_df_meal = pd.DataFrame(tvar_list_meal) 
tvar_df_meal

,0
0,NaN
1,2416.092473
2,2642.531183
3,904.632258
4,1462.333333
...,...
598,2577.449462
599,1463.980645
600,954.950724
601,1616.782796


In [67]:
def kurtosis_c(meal_list,row):
  result = kurtosis(meal_list[row])
  return (result)

kurtosis_list_meal = []
for row in range (len(meal_cgm_master_list)):
  kurtosis_val = kurtosis_c(meal_cgm_master_list,row)
  kurtosis_list_meal.append(kurtosis_val)
kurtosis_df_meal = pd.DataFrame(kurtosis_list_meal) 
kurtosis_df_meal

,0
0,NaN
1,0.537228
2,14.819425
3,0.005602
4,17.603705
...,...
598,0.348246
599,19.909529
600,13.146777
601,3.539182


In [68]:
def coeff_of_var(meal_data,row):
  mean = np.mean(meal_data[row])
  std_dev = np.std(meal_data[row])
  result = mean/std_dev
  return (result)

coeff_of_var_meal_list = []
for row in range(len(meal_cgm_master_list)):
    coeff_val = coeff_of_var(meal_cgm_master_list,row)
    coeff_of_var_meal_list.append(coeff_val)
coeff_of_var_df_meal = pd.DataFrame(coeff_of_var_meal_list) 
coeff_of_var_df_meal 

,0
0,NaN
1,2.971337
2,4.899025
3,2.668910
4,5.077272
...,...
598,2.820633
599,5.213251
600,4.745929
601,3.788889


In [69]:
def discrete_wavelet_trans(meal_data,row):
  (cA,cD) = pywt.dwt(meal_data[row], 'db1')
  cA = cA[::-1][0:8]
  return (cA,cD)

x_meal=[]
for row in range(len(meal_cgm_master_list)):
  x1,y1 = discrete_wavelet_trans(meal_cgm_master_list,row)
  x_meal.append(list(x1))
  discrete_wavelet_df_meal = pd.DataFrame(x_meal)  
discrete_wavelet_df_meal 

,0,1,2,3,4,5,6,7
0,4.242641,NaN,NaN,NaN,412.950360,403.050865,393.858477,417.900108
1,8.485281,282.135606,277.892965,264.457936,245.366053,234.759451,234.052345,231.223917
2,4.242641,396.686904,394.565584,377.595021,383.958982,386.787409,366.281313,336.582828
3,4.242641,111.722871,99.702056,100.409163,105.358910,97.580736,95.459415,117.379726
4,2.828427,296.277741,296.277741,299.106168,302.641702,299.106168,288.499567,262.336616
...,...,...,...,...,...,...,...,...
598,1.414214,282.135606,265.872150,284.964033,294.156421,241.830519,219.203102,191.625938
599,2.828427,299.106168,295.570635,282.135606,261.629509,257.386868,256.679762,282.135606
600,1.414214,150.613744,177.483802,200.818326,202.939646,203.438780,206.932720,210.426659
601,5.656854,262.336616,283.549819,248.194480,239.709199,243.951840,235.466558,246.073160


In [70]:
def windowed_mean(meal_list,row):
  w_size = 5
  window_averages =[]
  i=0
  while i < len(meal_list[row]) - w_size +1: 
    nw_size = meal_list[row][i : i + w_size]
    w_average = sum(nw_size)/w_size
    window_averages.append(w_average)
    i+=w_size
  # print(len(window_averages))
  return (window_averages[1:5]) 

windowed_average_list_meal = []
for row in range (len(meal_cgm_master_list)):
  average_list = windowed_mean(meal_cgm_master_list,row)
  windowed_average_list_meal.append(average_list)
windowed_average_list_df_meal = pd.DataFrame(windowed_average_list_meal)
windowed_average_list_df_meal

,0,1,2,3
0,315.000000,315.600000,291.600000,NaN
1,134.600000,157.000000,162.600000,172.600000
2,239.000000,222.600000,244.400000,271.400000
3,90.600000,126.200000,82.600000,71.800000
4,193.400000,172.800000,190.200000,212.800000
...,...,...,...,...
598,129.000000,142.600000,143.400000,192.800000
599,208.800000,201.600000,193.800000,186.000000
600,160.529412,154.352941,148.176471,144.341176
601,128.000000,158.400000,172.600000,169.200000


In [71]:
def calc_peaks_fouriertrans(meal_list,row):
    fouriertrans = list()
    peak_feat = list()
    arr = np.array(meal_list[row])
    # print(fft(arr))
    fouriertrans.append(abs(fft(arr)))
    for val in range(len(fouriertrans)):
      sets = set(fouriertrans[val])  
      set_list = list(sets)
      set_list.sort()
      set_list = set_list[::-1][0:8]
      # print(set_list)
      peak_feat+=set_list
    # print(peak_feat)
    return (fouriertrans,peak_feat)  


x_meal=[]
y_meal=[]
for row in range(len(meal_cgm_master_list)):
  x1,y1 = calc_peaks_fouriertrans(meal_cgm_master_list,row)
  x_meal.append(x1)
  y_meal.append(list(y1))
# print(len(y_meal))
peak_valdf_meal = pd.DataFrame(y_meal)
peak_valdf_meal

,0,1,2,3,4,5,6,7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4454.000000,664.456337,572.601635,336.628666,193.551452,175.570906,171.695251,144.303987
2,7680.000000,320.826129,300.013647,299.140088,297.346464,296.001831,294.701776,292.034000
3,2448.000000,498.109168,340.013615,95.348975,79.209519,74.201305,72.336252,68.159329
4,5921.000000,324.165126,224.411009,220.053890,215.919667,214.511978,210.804808,209.324877
...,...,...,...,...,...,...,...,...
598,4367.000000,681.374068,625.492950,234.901196,204.282845,188.377162,182.704244,178.605642
599,6083.000000,327.769970,252.928750,235.963234,216.214807,211.847581,207.960863,205.826880
600,4472.529412,300.734435,250.372124,220.375718,179.360386,144.090004,143.934188,143.451950
601,4646.000000,559.896486,315.203271,251.868256,218.193221,187.617157,164.811388,153.835998


In [72]:
#def rank1(meal_cgm_master_list):
def calc_rank_row(meal_data,row):
  rank = [0 for x in range(len(meal_data[row]))]
  for j in range (len(meal_data[row])):
    (r,s)=(1,1)
    for k in range (len(meal_data[row])):
      if k != j and meal_data[row][k] < meal_data[row][j]:
        r += 1
      if k != j and meal_data[row][k] == meal_data[row][j]:
        s += 1       
    rank[j] = r + (s - 1) / 2
  return (rank) 

rank_list_meal=[]
for row in range(len(meal_cgm_master_list)):
    ranks_list_meal = calc_rank_row(meal_cgm_master_list,row)
    rank_list_meal.append(ranks_list_meal)

ranks_list_df_meal = pd.DataFrame(rank_list_meal)
ranks_list_df_meal = ranks_list_df_meal.iloc[:,:24]
ranks_list_df_meal

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,19.0,11.5,10.0,14.0,14.0,14.0,16.5,16.5,22.5,25.0,24.0,21.0,19.0,22.5,19.0,11.5,9.0,7.0,3.0,2.0,4.0,5.0,8.0,6.0
1,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,12.5,15.5,17.5,15.5,12.5,11.0,14.0,17.5,19.5,21.0,22.0,23.0,19.5,24.0,25.0
2,31.0,30.0,27.0,20.0,16.0,14.0,13.0,11.0,9.0,2.0,4.0,4.0,4.0,6.0,7.0,8.0,10.0,12.0,15.0,17.0,22.0,25.0,23.5,21.0
3,3.5,3.5,3.5,3.5,6.0,12.5,21.0,22.0,24.0,25.5,27.0,29.0,30.0,31.0,28.0,25.5,23.0,18.5,10.0,7.0,8.5,12.5,17.0,15.5
4,27.5,24.0,18.0,14.0,13.0,15.5,15.5,12.0,10.0,9.0,8.0,6.0,4.0,2.0,3.0,5.0,7.0,11.0,17.0,19.0,24.0,29.5,27.5,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.5,16.0,18.0,14.5,14.5,20.0,19.0,12.5,10.5,12.5,17.0,21.5,21.5,23.0,30.0,31.0
599,21.0,28.0,30.5,30.5,26.0,27.0,24.5,22.0,15.0,13.5,12.0,10.5,8.5,13.5,29.0,19.0,17.0,8.5,6.0,2.0,3.5,5.0,3.5,7.0
600,31.0,30.0,29.0,28.0,27.0,26.0,25.0,24.0,23.0,22.0,21.0,20.0,19.0,18.0,17.0,16.0,15.0,14.0,12.0,11.0,9.0,8.0,7.0,10.0
601,4.0,5.0,2.0,3.0,7.0,10.5,9.0,8.0,6.0,10.5,12.0,13.0,14.0,21.0,25.0,27.0,26.0,19.0,16.5,18.0,20.0,23.0,22.0,16.5


In [73]:
def z_score_feat(meal_list,row):
  z_score_arr = np.array(meal_list[row])
  z_score_list = stats.zscore(z_score_arr)
  return (z_score_list)

z_scores_meal=[]
for row in range (len(meal_cgm_master_list)):
  z_score_res_list = z_score_feat(meal_cgm_master_list,row)
  z_scores_meal.append(z_score_res_list)
z_score_res_list_df_meal = pd.DataFrame(z_scores_meal)
z_score_res_list_df_meal = z_score_res_list_df_meal.iloc[:,:24]
z_score_res_list_df_meal

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.771862,-1.751181,-1.668459,-1.503014,-1.296208,-0.861915,-0.262177,-0.076051,0.068713,0.192797,0.316880,0.399603,0.316880,0.192797,0.151435,0.234158,0.399603,0.420283,0.440964,0.461645,0.503006,0.420283,0.523687,0.709812
2,1.112487,0.875190,0.657669,0.400597,0.222625,0.143526,0.005103,-0.133320,-0.331067,-0.548589,-0.509039,-0.509039,-0.509039,-0.489265,-0.469490,-0.390391,-0.291518,-0.093770,0.163301,0.281949,0.440147,0.578570,0.519246,0.420372
3,-1.317011,-1.317011,-1.317011,-1.317011,-0.641062,-0.269290,0.136280,0.271470,0.710837,1.116407,1.420584,1.555774,1.724761,1.792356,1.488179,1.116407,0.339065,-0.066505,-0.336884,-0.438277,-0.404479,-0.269290,-0.134100,-0.167897
4,0.558234,0.505069,0.345573,0.239243,0.212661,0.265826,0.265826,0.079748,-0.079748,-0.212661,-0.265826,-0.398739,-0.558234,-0.611399,-0.584817,-0.505069,-0.318991,0.026583,0.318991,0.372156,0.505069,0.584817,0.558234,0.664564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,-1.679333,-1.659310,-1.579219,-1.338945,-0.738261,-0.618124,-0.297759,-0.157599,-0.117553,0.002584,0.062652,-0.077508,-0.077508,0.142743,0.122720,-0.097530,-0.117553,-0.097530,0.022606,0.543200,0.543200,0.663336,1.324089,1.364135
599,0.339380,0.605056,0.658191,0.658191,0.525353,0.551921,0.498786,0.392515,0.126839,0.100271,0.047136,-0.005999,-0.059134,0.100271,0.631624,0.286245,0.233109,-0.059134,-0.271675,-0.510784,-0.431081,-0.324811,-0.431081,-0.165405
600,0.819129,0.778494,0.737859,0.697224,0.656589,0.615954,0.575319,0.534684,0.494049,0.453414,0.412779,0.372144,0.331509,0.290874,0.250239,0.209604,0.168969,0.128334,0.087699,0.047064,0.006429,-0.034206,-0.074841,0.023844
601,-1.109102,-1.083821,-1.260788,-1.159664,-0.679325,-0.451796,-0.527639,-0.578201,-0.755168,-0.451796,-0.300110,-0.123143,0.256072,0.534163,0.711130,0.812254,0.761692,0.458320,0.407758,0.433039,0.483601,0.660568,0.584725,0.407758


In [74]:
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize 
from sklearn.decomposition import PCA
meal_data = pd.concat([difference_df_meal,coeff_of_var_df_meal,discrete_wavelet_df_meal,windowed_average_list_df_meal,peak_valdf_meal,ranks_list_df_meal,z_score_res_list_df_meal,kurtosis_df_meal,skew_df_meal,tvar_df_meal,median_absolute_deviation_df_meal],axis=1)
dataset = (meal_data.fillna(0)).reset_index().drop(columns = ['index'])
kmeans = KMeans(n_clusters=6)
#model = KMeans(n_clusters=5, init='random', max_iter=100, n_init=1, verbose=1)
cluster = kmeans.fit_predict(dataset)
centroids = kmeans.cluster_centers_
#print(dataset)
kmeans_sse=kmeans.inertia_
print(kmeans_sse)
cluster_list=[]
for i in cluster:
  i+=1
  cluster_list.append(int (i))
dict1=collections.Counter(cluster_list)
print(dict1)
dataset['clusters']=cluster_list
print(dataset)
df=meal_cgm_master['bins'].fillna(1)
ground_truth_list = df.tolist()
gt_bin_list=[]
for i in ground_truth_list:
    i=int (i)
    gt_bin_list.append(i)  
print(gt_bin_list)


624766168.617292
Counter({2: 198, 3: 187, 4: 98, 6: 64, 1: 47, 5: 9})
              0         0         0  ...            0          0  clusters
0    321.000000  0.000000  4.242641  ...     0.000000   0.000000         3
1    197.000000  2.971337  8.485281  ...  2416.092473  28.169400         2
2    301.000000  4.899025  4.242641  ...  2642.531183  28.169400         4
3    129.000000  2.668910  4.242641  ...   904.632258  19.273800         3
4    214.000000  5.077272  2.828427  ...  1462.333333  13.343400         2
..          ...       ...       ...  ...          ...        ...       ...
598  208.000000  2.820633  1.414214  ...  2577.449462  45.960600         2
599  219.000000  5.213251  2.828427  ...  1463.980645  13.343400         2
600  168.176471  4.745929  1.414214  ...   954.950724  12.820129         3
601  197.000000  3.788889  5.656854  ...  1616.782796  34.099800         2
602  128.000000  2.673350  1.414214  ...   879.666667  34.099800         3

[603 rows x 75 columns]
[3, 6

In [85]:
from sklearn.metrics import confusion_matrix
y_true = gt_bin_list
y_pred = cluster_list
x=confusion_matrix(y_true, y_pred)
entropy=[]
purity=[]
print(x)
for row in range(len(x)):
  lg_t = np.log((x[row,:]/sum(x[row,:]))/np.log(2))
  ent = sum((-x[row,:]/sum(x[row,:]))*lg_t)
  entropy.append(ent)
  pur = max(x[row,:])/sum(x[row,:])
  purity.append(pur)
print(entropy)
print(purity)
total_x = sum(x,1)
tot_purity1 = 0
tot_entropy_list=[]
tot_entropy1 = 0
for row in range (len(x)):
  sum_s = sum(x[row,:])/sum(total_x) 
  if(not np.isnan(sum_s*entropy[row])):
    # print(sum_s*entropy[row])                
    tot_entropy1 = tot_entropy1 + (sum_s*entropy[row])
  #print(sum_s*entropy[row])
  tot_purity1 = round(tot_purity1 + (sum_s*purity[row]),4)  
print(tot_purity1)  
print(tot_entropy1)



[[16 63 65 37  1 13]
 [12 58 41 31  2 16]
 [ 8 38 49 22  4 21]
 [ 5 22 20  6  2 10]
 [ 5 15 10  2  0  3]
 [ 1  2  2  0  0  1]]
[1.0928333852817964, 1.147521229652032, 1.1875811537730017, 1.1751440244899427, nan, nan]
[0.3333333333333333, 0.3625, 0.34507042253521125, 0.3384615384615385, 0.42857142857142855, 0.3333333333333333]
0.3464
1.0537385752083546


In [76]:
from sklearn.cluster import DBSCAN 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import normalize 
from sklearn.decomposition import PCA

scaler = StandardScaler() 
dataset_scaled = scaler.fit_transform(dataset.drop(columns = ['clusters'])) 
dataset_normalized = normalize(dataset_scaled)
dataset_normalized = pd.DataFrame(dataset_normalized)
print(dataset_normalized)
pca = PCA(n_components = 7) 
dataset_principal = pca.fit_transform(dataset_normalized) 
dataset_principal = pd.DataFrame(dataset_principal) 
dataset_principal.columns = ['P1', 'P2','P3','P4','P5','P6','P7'] 
#print(dataset_principal.head())
db_clusters = DBSCAN(eps = 0.3999, min_samples =6).fit(dataset_principal) 
db_labels = db_clusters.labels_
dict1=collections.Counter(db_labels)
print(db_labels)
print(dict1)

           0         1         2   ...        71        72        73
0    0.120697 -0.301957  0.038208  ...  0.102279 -0.089297 -0.104113
1   -0.045319 -0.129212  0.428851  ...  0.071329 -0.003372 -0.053717
2    0.122574  0.113819  0.047976  ... -0.125476  0.008221 -0.033323
3   -0.123779 -0.108090  0.046892  ...  0.119549 -0.069324 -0.062635
4   -0.004162  0.157474 -0.029535  ... -0.192446 -0.054415 -0.101101
..        ...       ...       ...  ...       ...       ...       ...
598 -0.018461 -0.144527 -0.147676  ...  0.110096  0.007970  0.042751
599  0.004426  0.171081 -0.029068  ... -0.217320 -0.053467 -0.099504
600 -0.100483  0.141519 -0.140060  ... -0.166597 -0.098697 -0.124242
601 -0.058076  0.004359  0.249261  ...  0.035205 -0.079519 -0.026476
602 -0.127052 -0.109250 -0.096610  ...  0.105046 -0.071480 -0.012713

[603 rows x 74 columns]
[-1 -1 -1 -1 -1  0  0  2  0  1  0 -1  0  2  2  2  2 -1  0 -1  0  1 -1  2
  0  0  0  0  2 -1 -1  0 -1  0  0  2  2  2  0 -1  0  0  2  2  2  3  0 -1
 

In [77]:
centroids_list = centroids
print(len(centroids_list[0]))
centroid_dict = {i:centroids_list[i] for i in range(0,len(centroids_list))}
dataset_normalized['dblabels']=db_labels
# dataset_normalized.loc[0,'dblabels']
arr=[]
for idx,row in dataset_normalized.iterrows():
  if row.loc['dblabels']==-1.0:
    
    point = np.array(list(row)[:-1])
    # print(point,len(point))
    distances = []
    for j in centroid_dict:
      distances.append(np.linalg.norm(point-centroid_dict[j]))
    print(distances)
    dataset_normalized.at[idx,'dblabels'] = distances.index(min(distances))

    # df = DataFrame(row)
    # df=df.values.tolist()
    # print(df)
 
print(dataset_normalized)  
  


74
[9639.681126296831, 5520.266109775435, 3665.2227537969557, 7822.314056058, 13465.780318749525, 6326.422533782571]
[9639.427434894716, 5520.004610904639, 3664.9594432913077, 7822.060935668671, 13465.553152820963, 6326.155462211667]
[9639.226086458526, 5519.73690297441, 3664.6841016870785, 7821.802138896924, 13465.403144357417, 6325.93394888602]
[9639.632907806039, 5520.222389269502, 3665.177754886389, 7822.272124913369, 13465.731186108304, 6326.371518562061]
[9639.389119715059, 5519.909548328974, 3664.85718492968, 7821.9725268826405, 13465.54764523396, 6326.104443951691]
[9639.496943293847, 5520.029501186169, 3664.975655881618, 7822.08941599801, 13465.636592785033, 6326.217416491635]
[9639.672519573045, 5520.254509823779, 3665.2094974089396, 7822.3038527236395, 13465.771930362294, 6326.413358797741]
[9639.44225910917, 5519.975547688772, 3664.9245670609057, 7822.03756998175, 13465.59053732738, 6326.16126012966]
[9639.522070661716, 5520.051188530819, 3664.9973487926454, 7822.1136837197

In [78]:
def dbscan_sse(centroids, meal_data_val):
  cluster_sse = 0
  for row in meal_data_val:
    sse_eachrow = 0
    for i in range(len(row)):
      sse_eachrow = sse_eachrow + math.pow((centroids[i] - row[i]),2)
      cluster_sse = cluster_sse + sse_eachrow 
  return cluster_sse
def dbscan_func_sse(nums, centroids, dataset):
  dbscan_sse_val = 0
  for j in range(0,6):
    meal_data_val= dataset.loc[dataset['dblabels']==j].iloc[:,0:6]
    meal_arr = np.array(meal_data_val)
    dbscan_sse_val += dbscan_sse(centroids[j], meal_arr)
  return dbscan_sse_val
sse_db_scan=dbscan_func_sse(6,centroids,dataset_normalized)
print(sse_db_scan)

420888483.9538448


In [80]:
dict1=collections.Counter(dataset_normalized['dblabels'])
dict1
list1=(list(dataset_normalized['dblabels']))

db_pred=[]
for i in list1:
  i+=1
  db_pred.append(i)

y_true = gt_bin_list
x=confusion_matrix(y_true, db_pred)
entropy=[]
purity=[]
print(x)
for row in range(len(x)):
  lg_t = np.log((x[row,:]/sum(x[row,:]))/np.log(2))
  ent = sum((-x[row,:]/sum(x[row,:]))*lg_t)
  entropy.append(ent)
  pur = max(x[row,:])/sum(x[row,:])
  purity.append(pur)
total_x = sum(x,1)
tot_purity = 0
tot_entropy_list=[]
tot_entropy = 0
for row in range (len(x)):
  sum_s = sum(x[row,:])/sum(total_x) 
  if(not np.isnan(sum_s*entropy[row])):
    tot_entropy = tot_entropy + (sum_s*entropy[row])
  #print(sum_s*entropy[row])
  tot_purity = round(tot_purity + (sum_s*purity[row]),4)
print(tot_entropy)   
print(tot_purity) 

[[ 65   1 124   3   1   1]
 [ 63   3  89   1   2   2]
 [ 68   2  67   0   2   3]
 [ 37   0  26   1   0   1]
 [ 13   1  21   0   0   0]
 [  1   0   5   0   0   0]]
0.2811661145312573
0.5649


In [87]:
fin_dataframe = DataFrame()
fin_dataframe['SSE for Kmeans']=[kmeans_sse]
fin_dataframe['SSE for dbscan']=[sse_db_scan]
fin_dataframe['Entropy for Kmeans']=[tot_entropy1]
fin_dataframe['Entropy for Dbscan']=[tot_entropy]
fin_dataframe['Purity for Kmeans']=[tot_purity1]
fin_dataframe['Purity for Dbscan']=[tot_purity]
print(fin_dataframe)

   SSE for Kmeans  SSE for dbscan  ...  Purity for Kmeans  Purity for Dbscan
0    6.247662e+08    4.208885e+08  ...             0.3464             0.5649

[1 rows x 6 columns]


In [89]:
fin_dataframe.to_csv('Results.csv', index=False)